In [ ]:
import pandas as pd
import altair as alt
import math
import numpy as np

In [ ]:
alt.data_transformers.disable_max_rows()

## 1. Load datasets and improve variable names

In [ ]:
series = pd.read_csv("../data/tv_series_over_1k_votes.csv", encoding="latin_1")

In [ ]:
episodes = pd.read_csv("../data/episodes_details.csv", encoding="latin_1")

In [ ]:
episodes_renaming = {
    "Title_show_name_tconst": "seriesId",	
    "Title_basics_tconst": "episodeId",	
    "Title_show_name_primaryTitle": "seriesTitle",	
    "Title_basics_primaryTitle": "episodeTitle",	
    "SeriesName": "seriesFullName",
    "averageRating": "episodeAverageRating",
    "genres": "episodeGenres",
    "seasonNumber": "episodeSeasonNumber"
}

series_renaming = {
    "tconst": "seriesId",
    "primaryTitle": "seriesTitle",
    "SeriesName": "seriesFullName",
    "averageRating": "seriesAverageRating",
    "genres": "seriesGenres",
    "numVotes": "seriesNumVotes",
    "startYear": "seriesStartYear"
}


series = series.rename(series_renaming, axis=1)
episodes = episodes.rename(episodes_renaming, axis=1)

In [ ]:
print(f"Number of series: {len(series)}")
print(f"Number of episodes: {len(episodes)}")

## 2. Clean the dataframes

In [ ]:
# Clean "\\N" characters in "episodeNumber" & "seasonNumber"
episodes["episodeSeasonNumber"] = episodes["episodeSeasonNumber"].replace(r"\\N",np.nan, regex=True)
episodes["episodeNumber"] = episodes["episodeNumber"].replace(r"\\N",np.nan, regex=True)

# Investigate missing values
series_with_nan = series[series.isna().any(axis=1)]
episodes_with_nan = episodes[episodes.isna().any(axis=1)]

episodes_with_nan.merge(series[["seriesId", "seriesNumVotes"]], on="seriesId", how="left", indicator=True).sort_values(by="seriesNumVotes", ascending=False)

In [ ]:
# Drop series with missing values (only 1)
series.dropna(inplace=True)

# Change startYear to int
series["seriesStartYear"] = series["seriesStartYear"].astype("int")
series.dtypes

In [ ]:
# Drop episodes with missing episode number and season number (following investigation: this mostly concerns specials)
episodes.dropna(inplace=True)
episodes["episodeSeasonNumber"] = episodes["episodeSeasonNumber"].astype("int")
episodes["episodeNumber"] = episodes["episodeNumber"].astype("int")

episodes.dtypes

## 3. Create new variables

In [ ]:
# Create decade
def get_decade(x):
    return str(x)[:-1] + "0"

series["seriesDecade"] = series["seriesStartYear"].apply(lambda x: get_decade(x))

In [ ]:
# Create continuous episode number
episodes["episodeContinuousNumber"] = episodes.sort_values(by=["seriesId", "episodeSeasonNumber", "episodeNumber"]).groupby(by=["seriesId"]).cumcount() + 1

In [ ]:
# Create total number of episodes & total number of seasons
# Variable is added back to episodes (and not series) because it would filter the series dataframe (see below)
max_episodes_seasons = episodes[["seriesId", "episodeContinuousNumber", "episodeSeasonNumber"]].groupby("seriesId").max().rename(columns={"episodeContinuousNumber": "seriesNumEpisodes", "episodeSeasonNumber": "seriesNumSeasons"})
episodes = episodes.merge(max_episodes_seasons, on="seriesId")

In [ ]:
# Create average episode rating
# Variable is added back to episodes (and not series) because it would filter the series dataframe (see below)
series_avg_episode_rating = episodes[["seriesId","episodeAverageRating"]].groupby("seriesId").mean().rename({"episodeAverageRating": "seriesAverageEpisodeRating"}, axis=1)
episodes = episodes.merge(series_avg_episode_rating, on="seriesId")

In [ ]:
episodes

## 4. Filtering & Exporting

In [ ]:
# Investigate series with no episodes
series_full = series.merge(max_episodes_seasons, on="seriesId", how="outer", indicator=True)
series_full.loc[series_full["_merge"] == "right_only"]#[["seriesId"]].merge(episodes, on="seriesId")

### Filtering episodes & series > Only keep those series with episodes (& episodes for which there are series entries)

In [ ]:
series_reduced = series[["seriesId", "seriesGenres", "seriesAverageRating", "seriesStartYear", "seriesDecade"]]

clean_data = episodes.merge(series_reduced, on="seriesId")
clean_data

In [ ]:
# filter data to top 100 series by number of votes
top_series = series.sort_values(by="seriesNumVotes", ascending=False)["seriesId"][0:100]
clean_data_reduced = clean_data.loc[clean_data["seriesId"].isin(top_series)]

# output episodes from top 100 series
clean_data_reduced.to_json("../app/src/clean_data.json", orient="records")

## Exploratory charts

In [ ]:
alt.Chart(series).mark_bar(tooltip=True).encode(
    alt.X("seriesNumVotes:Q"),
    alt.Y("seriesTitle:N", sort="-x"),
    color="seriesAverageRating:O"
).transform_window(rank="rank(seriesNumVotes)", sort=[alt.SortField("seriesNumVotes", order="descending")]).transform_filter(alt.datum.rank <= 200)

In [ ]:
alt.Chart(series).mark_circle(tooltip=True).transform_filter(
    "datum.seriesNumVotes >= 10000"
).encode(
    alt.X("seriesNumVotes:Q"),
    alt.Y("seriesAverageRating:Q"),
    alt.Color("seriesDecade:N"),
    opacity=alt.value(0.7)
).properties(width=600, height=600).interactive()

In [ ]:
alt.Chart(clean_data_reduced).mark_rect().encode(
    alt.X("episodeSeasonNumber:O"),
    alt.Y("seriesTitle:N"),
    alt.Color("mean(episodeAverageRating):Q")
)

In [ ]:
series_line = alt.Chart(clean_data_reduced).mark_line(color="purple", strokeWidth=1).encode(
    alt.X("episodeContinuousNumber:O"),
    alt.Y("seriesAverageRating:Q", axis=alt.Axis(values=[5])),
)

season_episode_average = alt.Chart(clean_data_reduced).mark_line(color="orange", strokeWidth=1).encode(
    alt.Y("seriesAverageEpisodeRating:Q", axis=alt.Axis(values=[5])),
    alt.X("episodeContinuousNumber:O"),
)

episodes_area = alt.Chart(clean_data_reduced).mark_area(opacity=0.5, color="black").encode(
    alt.X("episodeContinuousNumber:O"),
    alt.Y("episodeAverageRating:Q", axis=alt.Axis(values=[5])),
    tooltip=["episodeNumber", "episodeSeasonNumber"]
)

episodes_line = alt.Chart(clean_data_reduced).mark_line(color="black", strokeWidth=3).encode(
    alt.X("episodeContinuousNumber:O"),
    alt.Y("episodeAverageRating:Q", axis=alt.Axis(values=[5], title="rating")),
    tooltip=["episodeNumber", "episodeSeasonNumber"]
)


(series_line + season_episode_average + episodes_line).properties(
    width=1500, 
    height=100
    ).facet(
        row=alt.Row("seriesTitle", header=alt.Header(
            labelAngle=0, 
            labelFont="Arial",
            labelFontSize=18,
            labelAlign="left",
            labelPadding=40)),
        spacing=75,
        #data=episodes_top_rated_series
        ).resolve_scale(
            #y="independent"
            ).configure_axisBottom(
                title=None, 
                grid=False, 
                ticks=False,
                labels=False, 
                domain=True
                ).configure_axisLeft(
                    title=None, 
                    grid=True,
                    gridDash=[5, 5], 
                    gridColor="#BDBDBD",
                    domain=False,
                    labels=True,
                    ticks=False,
                    labelColor="#BDBDBD",
                    titleFont="Arial",
                    titleFontSize=12,
                    titleFontWeight=200,
                    titleAngle=0,
                    titleColor="#BDBDBD",
                    titleBaseline="top",
                    titleAnchor="end"
                    ).configure_view(
                        strokeOpacity=0)